# Livrable 1 - groupe 1

Nous sommes le groupe 1 composé de :  
- DELORME Alexandre
- ENCOGNERE Yanis
- MENNERON Laurine
- PEREON Alexandre
- ROCHARD Léo

## Contenu du livrable
TODO...


# TODO

##  Analyse des Résultats et Biais/VarianceAnalyse

Discussion sur le surapprentissage/sous-apprentissage

Calcul et affichage d'une matrice de confusion

Calcul de métriques supplémentaires (précision, rappel, F1-score)

## À implémenter/discuter:

Régularisation L2 dans les couches denses/convolutionnelles

Ajustement du taux de dropout

Utilisation de Batch Normalization

Transfer learning avec un modèle pré-entraîné (VGG16, ResNet, etc.)

Grid search pour optimiser les hyperparamètres

## Documentation et Justifications

Schéma de l'architecture du réseau (peut être généré avec tf.keras.utils.plot_model)

Justification des choix (architecture, hyperparamètres, etc.)

Analyse des résultats obtenus

Discussion sur les difficultés rencontrées (images réalistes dans les peintures)

## Gestion des Données Déséquilibrées

Analyse du déséquilibre entre classes photo/non-photo

Techniques pour gérer le déséquilibre (poids de classe, oversampling, etc.)

## EDA
TODO...
- Chargement du dataset
- Visualisation
- Répartition des données

### Chargement des bibliothèques

In [ ]:
import os
import PIL
import imghdr
import pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import gdown
import zipfile
import pathlib

In [ ]:
# ID du fichier (extrait de l'URL)
file_id = "1Q5QttYirEHvto38l6e6uk66Fjvuk_V7I"
# dataset_path = "dataset_livrable_1"
dataset_path = "dataset"
zip_path = dataset_path + ".zip"
extract_dir = pathlib.Path(zip_path).parent / dataset_path

# Vérifier si le dossier existe déjà
if not os.path.exists(extract_dir):
    print(f"Le dossier '{extract_dir}' n'existe pas. Téléchargement en cours...")
    gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_path, quiet=False)

    # Extraction du fichier ZIP
    print(f"Extraction ZIP en cours...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extraction Zip terminée")
else:
    print(f"Le dossier '{extract_dir}' existe déjà. Téléchargement et extraction non nécessaires.")

data_dir = extract_dir
print(f"Dataset disponible dans : {data_dir}")


In [ ]:
categories = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
print(f"Catégories détectées : {categories}")

validation_split = 0.2
seed = 42

batch_size = 32
img_height = 180
img_width = 180

### Pré traitement des images

In [ ]:

from collections import defaultdict

# Suppresion des fichiers corrompus ou non images --------------------------------------------------------------------
def clean_images_dataset(dataset_path_arg):
    """
    Fonction pour nettoyer le dataset en supprimant les fichiers corrompus ou non images.
    """
    # Dictionnaire pour stocker le nombre d'images corrompues par classe
    corrupted_count_by_class = defaultdict(int)
    dataset_path = dataset_path_arg
    print("Début de la vérification des images ...")

    # Récupération de toutes les images pour calculer la progression
    all_files = []
    for root, dirs, files in os.walk(dataset_path):  # Utilisation de os.walk pour parcourir récursivement
        for file_name in files:
            dir_name = os.path.basename(root)  # Nom du dossier courant
            all_files.append((dir_name, root, file_name))

    total_files = len(all_files)
    checked_files = 0  # Pour la progression

    # Parcours des images avec affichage de la progression
    for dir_name, dir_path, file_name in all_files:
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            try:
                with open(os.path.join(dir_path, file_name), 'rb') as file:
                    img_bytes = file.read()  # Lire les bytes de l'image
                    img = tf.image.decode_image(img_bytes)  # Essayer de décoder l'image
            except Exception as e:
                corrupted_count_by_class[dir_name] += 1
                print(f"\nImage corrompue : {file_name} dans {dir_name}. Exception: {e}")
                os.remove(os.path.join(dir_path, file_name))
                print(f"Image {file_name} supprimée.")
        else:
            corrupted_count_by_class[dir_name] += 1
            print(f"\nLe fichier {file_name} dans {dir_name} n'est pas une image.")
            os.remove(os.path.join(dir_path, file_name))
            print(f"Fichier {file_name} supprimé.")

        # Mise à jour de la progression
        checked_files += 1
        progress = (checked_files / total_files) * 100
        print(f"\rProgression : [{int(progress)}%] {checked_files}/{total_files} images vérifiées", end="")

    print("\nVérification des fichiers terminée.")

    # Affichage du nombre d'images corrompues par dossier
    for dir_name, count in corrupted_count_by_class.items():
        print(f"Dossier {dir_name} : {count} images corrompues")

    # Nombre total d'images corrompues
    total_corrupted = sum(corrupted_count_by_class.values())
    print(f"Nombre total d'images corrompues ou non image : {total_corrupted}")

In [ ]:
clean_images_dataset(data_dir)

### Chargement des images

In [ ]:
train_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=categories)

val_set = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=categories)

class_names = train_set.class_names
num_classes = len(class_names)

print(f"Classes found: {class_names}")

### Visualisation

In [ ]:
plt.figure(figsize=(8, 8))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

### Répartition des classes

In [ ]:
def count_images_foreach_label(dataset):
    label_counts = Counter(class_names[label] for _, labels in dataset for label in labels)
    return dict(label_counts)

# Count labels in train and validation sets
train_label_counts = count_images_foreach_label(train_set)
val_label_counts = count_images_foreach_label(val_set)

# Extract data for plotting
labels, train_counts = zip(*train_label_counts.items())

# Print counts
print(train_label_counts)

# Plot Train Set Distribution
plt.figure(figsize=(8, 4))
plt.bar(labels, train_counts)
plt.xticks(rotation=45)
plt.title('Train Set')
plt.xlabel('Labels')
plt.ylabel('Counts')
plt.tight_layout()
plt.show()


## Modélisation

### Optimisation pour l'entrainement

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_set = val_set.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from collections import Counter
import numpy as np
import tensorflow as tf

# Étape 1 : Calculer les poids pour chaque classe
def compute_class_weights(dataset, class_names):
    """
    Calcule les poids pour chaque classe en fonction de leur fréquence dans le dataset.
    """
    label_counts = Counter(class_names[label] for _, labels in dataset for label in labels)
    total_samples = sum(label_counts.values())
    class_weights = {class_name: total_samples / count for class_name, count in label_counts.items()}
    return class_weights

# Calcul des poids pour l'ensemble d'entraînement
class_weights = compute_class_weights(train_set, class_names)

# Normaliser les poids pour éviter des valeurs trop grandes
max_weight = max(class_weights.values())
class_weights = {class_name: weight / max_weight for class_name, weight in class_weights.items()}

print("Class Weights:", class_weights)

# Étape 2 : Appliquer les poids dans la fonction de perte
# Exemple : Si vous utilisez une perte catégorielle
class_weight_tensor = tf.constant([class_weights[class_name] for class_name in class_names], dtype=tf.float32)

def weighted_loss(y_true, y_pred):
    """
    Fonction de perte pondérée par classe pour des étiquettes entières.
    """
    weights = tf.gather(class_weight_tensor, tf.cast(y_true, tf.int32))
    unweighted_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return unweighted_loss * weights


### Définition du modèle

In [ ]:
# Le modèle
model = Sequential()

# Data augmentation
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

# Ajout des couches au modèle
model.add(data_augmentation)
model.add(layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))  # Ajout de la couche Dropout
model.add(layers.Dense(num_classes))

# Compilation du modèle
# model.compile(optimizer = 'adam',
#               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               # loss=weighted_categorical_crossentropy,
#               metrics=['accuracy'])
model.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy'])

model.summary()

### Entrainement

#### Définition du early callback 

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./L1_model.keras', save_best_only=True)

#### Fit

In [ ]:
epochs=4

history = model.fit(train_set, 
          epochs=epochs,
          batch_size=batch_size,
          validation_data=val_set,
          verbose=1, 
          callbacks=[early_stopping, model_checkpoint])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()